<a href="https://colab.research.google.com/github/WILLIANSCASACOLA/Meus-Codigos/blob/main/Pesquisar_Cep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import requests
import json
from pyspark.sql import SparkSession

# Inicia a sessão Spark
spark = SparkSession.builder.appName("ViaCEP").getOrCreate()

In [51]:
# Pede o CEP ao usuário
cep = input("Digite o CEP (apenas números): ")

Digite o CEP (apenas números): 06663330


In [52]:

# Monta a URL com o CEP informado
url = f"https://viacep.com.br/ws/{cep}/json/"

response = requests.get(url)

# Verifica se a resposta foi bem-sucedida
if response.status_code == 200:
    data = response.json()

    # Transforma o dicionário em uma lista de um item (Spark espera uma lista de registros)
    data_list = [data]

    # Cria o DataFrame
    df = spark.createDataFrame(data_list)

    # Mostra o resultado
    df.show(truncate=False)
else:
    print(f"Erro ao acessar a API: {response.status_code}")

+---------------+---------+-----------+---+---------+----+-------+----------+-----------------------+-------+-----+---+-------+
|bairro         |cep      |complemento|ddd|estado   |gia |ibge   |localidade|logradouro             |regiao |siafi|uf |unidade|
+---------------+---------+-----------+---+---------+----+-------+----------+-----------------------+-------+-----+---+-------+
|Jardim Paulista|06663-330|           |11 |São Paulo|3736|3522505|Itapevi   |Rua José de Paula Gomes|Sudeste|6551 |SP |       |
+---------------+---------+-----------+---+---------+----+-------+----------+-----------------------+-------+-----+---+-------+



In [53]:
# exportar para csv o arquivo df convertido em df pandas
df_pandas = df.toPandas()
df_pandas.to_csv(f'viacep_{cep}.csv', index=False, encoding='utf-8-sig')

In [54]:
print("Arquivo CSV gerado com sucesso!")

Arquivo CSV gerado com sucesso!


In [55]:
df_pandas

,bairro,cep,complemento,ddd,estado,gia,ibge,localidade,logradouro,regiao,siafi,uf,unidade
0,Jardim Paulista,06663-330,,11,São Paulo,3736,3522505,Itapevi,Rua José de Paula Gomes,Sudeste,6551,SP,
